In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from tqdm import tqdm
import shutil
import random
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.layers import  Dropout, Dense, Flatten, BatchNormalization,Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv("/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv")
df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


Preparing the data

In [3]:
df[df["O"]==1]

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
5,6,60,Male,6_left.jpg,6_right.jpg,macular epiretinal membrane,moderate non proliferative retinopathy，epireti...,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",6_right.jpg
6,7,60,Female,7_left.jpg,7_right.jpg,drusen,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",7_right.jpg
8,9,54,Male,9_left.jpg,9_right.jpg,normal fundus,vitreous degeneration,0,0,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['O'],"[0, 0, 0, 0, 0, 0, 0, 1]",9_right.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5984,4219,63,Male,4219_left.jpg,4219_right.jpg,normal fundus,post retinal laser surgery,0,0,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",4219_left.jpg
6007,4243,56,Female,4243_left.jpg,4243_right.jpg,severe nonproliferative retinopathy,post retinal laser surgery，moderate non prolif...,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4243_left.jpg
6276,4551,53,Male,4551_left.jpg,4551_right.jpg,mild nonproliferative retinopathy,intraretinal microvascular abnormality,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4551_left.jpg
6310,4588,64,Female,4588_left.jpg,4588_right.jpg,mild nonproliferative retinopathy,macular epiretinal membrane,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4588_left.jpg


In [3]:
train=df.drop(columns=['M', 'D', 'H', ],axis=1)


train=train[((train['G'] == 1) | (train['C'] == 1) | (train['A']== 1) | (train['N'] == 1)| (train['O']==1))]

# Classes
classes=['N','G', 'C', 'A', 'O']


train = train.reset_index(drop=True)

In [6]:
train.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,G,C,A,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,1,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,6,60,Male,6_left.jpg,6_right.jpg,macular epiretinal membrane,moderate non proliferative retinopathy，epireti...,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",6_right.jpg


Creating a list of each class

In [4]:
glaucoma=[]
for i in range(len(train)):
    if(train['G'][i]==1):
        glaucoma.append(train['filename'][i])
        
cataract=[]
for i in range(len(train)):
    if(train['C'][i]==1):
        cataract.append(train['filename'][i])

amd=[]
for i in range(len(train)):
    if(train['A'][i]==1):
        amd.append(train['filename'][i])
        
normal=[]
for i in range(len(train)):
    if(train['N'][i]==1):
        normal.append(train['filename'][i])
        
other=[]
for i in range(len(train)):
    if(train['O'][i]==1):
        other.append(train['filename'][i])       

In [5]:
base_dir = './Dataset/'
if os.path.isdir(base_dir) is False:
    os.mkdir(base_dir)

#Train, Testing and Validation set
train_dir = os.path.join(base_dir, 'Training')
os.mkdir(train_dir)

testing_dir = os.path.join(base_dir, 'Testing')
os.mkdir(testing_dir)

validation_dir = os.path.join(base_dir, 'Validation')
os.mkdir(validation_dir)

# Under train folder create four folders 
# (Glaucoma, Cataract, Age related Macular Degeneration, Normal, Others)

train_glaucoma_dir = os.path.join(train_dir, 'glaucoma')
os.mkdir(train_glaucoma_dir)

train_cataract_dir = os.path.join(train_dir, 'cataract')
os.mkdir(train_cataract_dir)

train_age_macular_dir = os.path.join(train_dir, 'age_related_macular_degeneration')
os.mkdir(train_age_macular_dir)

train_normal_dir = os.path.join(train_dir, 'normal')
os.mkdir(train_normal_dir)


train_other_diseases_dir = os.path.join(train_dir, 'other_diseases')
os.mkdir(train_other_diseases_dir)


# Under testing folder create four folders 

testing_glaucoma_dir = os.path.join(testing_dir, 'glaucoma')
os.mkdir(testing_glaucoma_dir)

testing_cataract_dir = os.path.join(testing_dir, 'cataract')
os.mkdir(testing_cataract_dir)

testing_age_macular_dir = os.path.join(testing_dir, 'age_related_macular_degeneration')
os.mkdir(testing_age_macular_dir)

testing_normal_dir = os.path.join(testing_dir, 'normal')
os.mkdir(testing_normal_dir)

testing_other_diseases_dir = os.path.join(testing_dir, 'other_diseases')
os.mkdir(testing_other_diseases_dir)

# Under validation folder create four folders 
# (Glaucoma, Cataract, Age related Macular Degeneration, Pathological Myopia)

validation_glaucoma_dir = os.path.join(validation_dir, 'glaucoma')
os.mkdir(validation_glaucoma_dir)

validation_cataract_dir = os.path.join(validation_dir, 'cataract')
os.mkdir(validation_cataract_dir)

validation_age_macular_dir = os.path.join(validation_dir, 'age_related_macular_degeneration')
os.mkdir(validation_age_macular_dir)

validation_normal_dir = os.path.join(validation_dir, 'normal')
os.mkdir(validation_normal_dir)

validation_other_diseases_dir = os.path.join(validation_dir, 'other_diseases')
os.mkdir(validation_other_diseases_dir)

Train test Split

In [6]:
glaucoma_source_dir = '/kaggle/input/ocr-filtred/Data/glaucoma/'
training_glaucoma_dir = './Dataset/Training/glaucoma/'
testing_glaucoma_dir = './Dataset/Testing/glaucoma/'
valid_glaucoma_dir = './Dataset/Validation/glaucoma/'

cataract_source_dir = '/kaggle/input/ocr-filtred/Data/cataract/'
training_cataract_dir = './Dataset/Training/cataract/'
testing_cataract_dir = './Dataset/Testing/cataract/'
valid_cataract_dir = './Dataset/Validation/cataract/'

age_related_macular_degeneration_source_dir = '/kaggle/input/ocr-filtred/Data/age_related_macular_degeneration/'
training_age_related_macular_degeneration_dir = './Dataset/Training/age_related_macular_degeneration/'
testing_age_related_macular_degeneration_dir = './Dataset/Testing/age_related_macular_degeneration/'
valid_age_related_macular_degeneration_dir = './Dataset/Validation/age_related_macular_degeneration/'

normal_source_dir = '/kaggle/input/ocr-filtred/Data/normal/'
training_normal_dir = './Dataset/Training/normal/'
testing_normal_dir = './Dataset/Testing/normal/'
valid_normal_dir = './Dataset/Validation/normal/'


other_diseases_source_dir = '/kaggle/input/ocr-filtred/Data/other_diseases/'
training_other_diseases_dir = './Dataset/Training/other_diseases/'
testing_other_diseases_dir = './Dataset/Testing/other_diseases/'
valid_other_diseases_dir = './Dataset/Validation/other_diseases/'

In [7]:
def split_data(SOURCE, TRAINING, TESTING, VALIDATION, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")
    
    SIZE = (1 - SPLIT_SIZE) / 2
    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) * SIZE)
    valid_length = int(len(files) * SIZE)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[training_length:training_length+testing_length]
    valid_set = shuffled_set[training_length+testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        shutil.copy(this_file, destination)
        
    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        shutil.copy(this_file, destination)
    
    for filename in valid_set:
        this_file = SOURCE + filename
        destination = VALIDATION + filename
        shutil.copy(this_file, destination)

In [8]:
split_size = 0.80

split_data(glaucoma_source_dir, training_glaucoma_dir, testing_glaucoma_dir, valid_glaucoma_dir, split_size)
split_data(cataract_source_dir, training_cataract_dir, testing_cataract_dir, valid_cataract_dir, split_size)
split_data(age_related_macular_degeneration_source_dir, training_age_related_macular_degeneration_dir, testing_age_related_macular_degeneration_dir, valid_age_related_macular_degeneration_dir, split_size)
split_data(other_diseases_source_dir, training_other_diseases_dir, testing_other_diseases_dir, valid_other_diseases_dir, split_size)
split_data(normal_source_dir, training_normal_dir, testing_normal_dir, valid_normal_dir, split_size)


In [9]:
def img_aug(img_height,img_width):
    train_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                        rotation_range = 30,
                                        width_shift_range = 0.1,
                                        height_shift_range = 0.1,
                                        zoom_range = 0.1,
                                        horizontal_flip = True,
                                       vertical_flip=True,
                                       shear_range=0.2,
                                brightness_range=[0.3,1]
                                       )
    train_generator = train_datagen.flow_from_directory('./Dataset/Training/', 
                                                        target_size=(img_height,img_width), 
                                                        class_mode='categorical')
    validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
    validation_generator = validation_datagen.flow_from_directory('./Dataset/Validation/', 
                                                              target_size=(img_height,img_width),
                                                              class_mode='categorical')
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_generator = test_datagen.flow_from_directory('./Dataset/Testing/', 
                                                              target_size=(img_height,img_width),
                                                              class_mode='categorical')
    
    return train_generator,validation_generator,test_generator

In [10]:
train_generator,validation_generator,test_generator = img_aug(224,224)

Found 3843 images belonging to 5 classes.
Found 486 images belonging to 5 classes.
Found 478 images belonging to 5 classes.


RandomizedSearchCV

<h3> VGG16 MODEL</h3>

In [ ]:
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import random
from keras.applications.vgg16 import VGG16, preprocess_input


# Set the input shape of the images
input_shape = (224, 224, 3)

# Define the base model with pretrained weights
base_model = VGG16(weights="/kaggle/input/tf-keras-pretrained-model-weights/No Top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False, input_shape=input_shape)

# Define the top layers of the model using Functional API
inputs = Input(shape=input_shape)
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(5, activation='softmax')(x)

# Define the number of epochs for training
epochs = 10

# Define the batch size for training
batch_size = 32

# Define the hyperparameters to search over
optimizers = [Adam, RMSprop, SGD]
learning_rates = [0.001, 0.0001]
units = [512, 1024, 4096]

# Set the number of random combinations to try
num_combinations = 10

best_score = float('-inf')
best_params = {}

# Perform a random search over the hyperparameters
for i in range(num_combinations):
    # Choose random values for the hyperparameters
    optimizer = random.choice(optimizers)
    learning_rate = random.choice(learning_rates)
    unit = random.choice(units)

    # Create the Keras model from the functional API model
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model with the chosen optimizer and learning rate
    opt = optimizer(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy')

    # Train the model
    history = model.fit(train_generator, batch_size=batch_size, epochs=epochs, validation_data=validation_generator, verbose=1)

    # Evaluate the model on the validation set
    score = model.evaluate(validation_generator, verbose=1)

    # Check if this is the best score
    if score > best_score:
        best_score = score
        best_params = {'optimizer': optimizer.__name__, 'learning_rate': learning_rate, 'units': unit}

# Print the best hyperparameters and score
print("Best parameters: ", best_params)
print("Best score: ", best_score)


In [17]:


# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='/kaggle/input/tf-keras-pretrained-model-weights/No Top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer to reduce the output feature map size
x = GlobalAveragePooling2D()(base_model.output)

# Add a fully connected layer with 1024 units and ReLU activation
x = Dense(1024, activation='relu')(x)

# Add a final softmax layer for classification
predictions = Dense(5, activation='softmax')(x)

# Set up the full model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=SGD(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [18]:

# early stopping
early_stop= EarlyStopping(monitor="val_accuracy",patience=50,min_delta=0.01,verbose=1)

# model check point 
filepath = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

model_check= ModelCheckpoint(filepath
                             ,monitor="val_accuracy",
                             min_delta=0.01,
                             verbose=1,
                             save_best_only=True,
                             mode="auto")

#saving model
model.save(filepath)
call_back=[early_stop,model_check]

batch_size=32
spe=train_generator.samples//batch_size
vaspe=validation_generator.samples//batch_size

In [ ]:
hist1=model.fit(train_generator,
              validation_data=validation_generator,
            steps_per_epoch=spe,
               validation_steps=vaspe,
              epochs=100,
              verbose=1,
                 callbacks=call_back
              )